In [8]:
import copy

poligon1 = [[0,1,0], [1,1,1], [0,1,0]]
poligon2 = [[1,1], [1,0]]
poligon3 = [[1,1,1], [0,1,0]]

import generate
import pandas as pd
import numpy as np



matrix = [[0,0,0,0,0,0,0,0],
          [0,0,0,0,0,0,0,0],
          [0,0,0,0,0,0,0,0],
          [0,0,0,0,0,0,0,0],
          [0,0,0,0,0,0,0,0],
          [0,0,0,0,0,0,0,0],
          [0,0,0,0,0,0,0,0]]
pallets = []

In [9]:

print_matrix(matrix)


In [10]:
def fit_item(pallet, item):
    for i in range(len(pallet[0])):
        for j in range(len(pallet)):
            exit = False
            # условия, что объект влезает
            if len(item) + i <= len(pallet) and len(item[0]) + j <= len(pallet[0]):
                # располагаем объект
                for p in range(len(item)):
                    for k in range(len(item[0])):
                        # решаем есть ли пересечение
                        if pallet[i+p][j+k] + item[p][k] == 2:
                            exit = True
                            break
                    if exit:
                        break
            else:
                exit = True
            # если пересечений нет и элемент влезает, то добавляем его
            if not exit:
                for p in range(len(item)):
                    for k in range(len(item[0])):
                        pallet[i+p][j+k] += item[p][k]
                break
        if not exit:
            break
    print_matrix(pallet)
    print('--------------')
    return pallet, exit

In [11]:
# fit_item(matrix, poligon2)

In [12]:
def fit_list(matrix, items):
    for item in items:
        matrix, exit = fit_item(matrix, item)
        # print_matrix(matrix)
        # print('--------------')
    return matrix

In [13]:
# fit_list(matrix, poligons)

In [14]:
def fit_pallets(matrix, pallets, poligons):
    pallets.append(copy.deepcopy(matrix))
    print_matrix(matrix)
    print('+++++++++++++++')

    for poligon in poligons:
        i=0
        exit = True
        while exit and i<len(pallets):
        # for pallet in pallets:
            pallets[i], exit = fit_item(pallets[i], poligon)
            if exit and i==(len(pallets)-1):
                pallets.append(copy.deepcopy(matrix))
            i+=1
    return pallets

In [15]:
fit_pallets(matrix, pallets, poligons)

[0, 0, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

+++++++++++++++
[0, 1, 0, 0, 0, 0, 0, 0] 

[1, 1, 1, 0, 0, 0, 0, 0] 

[0, 1, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

--------------
[0, 1, 0, 1, 1, 0, 0, 0] 

[1, 1, 1, 1, 0, 0, 0, 0] 

[0, 1, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

--------------
[0, 1, 0, 1, 1, 1, 1, 1] 

[1, 1, 1, 1, 0, 0, 1, 0] 

[0, 1, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

--------------
[0, 1, 0, 1, 1, 1, 1, 1] 

[1, 1, 1, 1, 1, 0, 1, 0] 

[0, 1, 0, 1, 1, 1, 0, 0] 

[0, 0, 0, 0, 1, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0, 0] 

[0, 0, 0, 0, 0, 0, 0,

[[[0, 1, 0, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 0, 1, 0],
  [0, 1, 0, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 0, 1, 0],
  [0, 1, 1, 1, 1, 1, 1, 0],
  [1, 1, 1, 1, 0, 1, 1, 1],
  [0, 1, 0, 0, 0, 0, 1, 0]],
 [[0, 1, 1, 1, 1, 0, 0, 0],
  [1, 1, 1, 1, 0, 0, 0, 0],
  [0, 1, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0]]]